# Import libraries

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow.
from keras.models import Sequential
from keras.layers import LSTM, Dense

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

# Fetch data

In [ ]:
stock_symbol = 'GOOGL'
start_date = '2020-01-01'
end_date = '2023-01-01'

# Fetch stock data using yfinance
stock_data = yf.download(stock_symbol, start=start_date, end=end_date, progress=False)
stock_data

In [ ]:
stock_data.tail()

# Preprocess data

In [ ]:
# Use 'Close' prices for prediction
data = stock_data['Close'].values.reshape(-1, 1)

# Scaling the data to values between 0 and 1
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

# Create time series train and test sets

In [ ]:
lookback = 60  # Number of previous days to consider
X, y = [], []
for i in range(len(data) - lookback):
    X.append(data[i:i+lookback, 0])
    y.append(data[i+lookback, 0])

X, y = np.array(X), np.array(y)
train_size = int(0.8 * len(X))

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build and train model

Un modelo LSTM (Long Short-Term Memory) es un tipo de red neuronal recurrente (RNN) diseñada para manejar secuencias y dependencias a largo plazo en datos. 

A diferencia de las RNN tradicionales, las LSTM están diseñadas para evitar el problema de la desaparición del gradiente, lo que les permite capturar relaciones a largo plazo en los datos de manera más efectiva.

Las LSTMs son especialmente útiles cuando se trabaja con datos secuenciales, como series de tiempo, texto o señales, donde la relación entre los puntos en la secuencia puede ser compleja y abarcar largos intervalos de tiempo.

La característica clave de las LSTMs es su capacidad para aprender y recordar información a largo plazo y olvidar información menos relevante. Esto se logra a través de unidades de memoria llamadas "celdas". Cada celda tiene tres puertas principales:

Puerta de olvido (Forget Gate): Esta puerta decide qué información pasada debe ser olvidada o ignorada en función de la entrada actual. Ayuda a la LSTM a recordar o descartar información importante de manera efectiva.

Puerta de entrada (Input Gate): Esta puerta determina qué nueva información debe ser almacenada en la celda de memoria. Se calcula en función de la entrada actual y la memoria anterior.

Puerta de salida (Output Gate): Esta puerta controla la salida de la celda de memoria. Filtra la información almacenada en la celda y produce la salida de la LSTM.

In [ ]:
model = Sequential()
model.add(LSTM(units=50, input_shape=(lookback, 1)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=50, batch_size=32)

## Model Architecture

In [ ]:
model.summary()

## Layer Information

In [ ]:
for layer in model.layers:
    print(layer.name, layer.trainable, layer.output_shape)

# Predictions

In [ ]:
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Evaluate model

In [ ]:
mse = mean_squared_error(y_test, y_pred)
mse

In [ ]:
test_dates = stock_data.index[train_size + lookback:].to_list()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(test_dates, y_test, label='Actual', color='black')
plt.plot(test_dates, y_pred, label='Predicted', color='red')
plt.legend()
plt.title(f'Predictions vs Actual\nLSTM MSE: {mse:.4f}')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.xticks(rotation=45)
plt.show()